# SQL Assignment

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import sqlite3

from IPython.display import display, HTML

In [3]:
# Note that this is not the same db we have used in course videos, please download from this link
# https://drive.google.com/file/d/1O-1-L1DdNxEK6O6nG2jS31MbrMh-OnXM/view?usp=sharing

In [4]:
conn = sqlite3.connect("/content/drive/MyDrive/Copy of Db-IMDB-Assignment.db")

#### Overview of all tables

In [5]:
tables = pd.read_sql_query("SELECT NAME AS 'Table_Name' FROM sqlite_master WHERE type='table'",conn)
tables = tables["Table_Name"].values.tolist()

In [42]:
for table in tables:
    query = "PRAGMA TABLE_INFO({})".format(table)
    schema = pd.read_sql_query(query,conn)
    print("Schema of",table)
    display(schema)
    print("-"*100)
    print("\n")

Schema of Movie


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,title,TEXT,0,None,0
3,3,year,TEXT,0,None,0
4,4,rating,REAL,0,None,0
5,5,num_votes,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of Genre


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,Name,TEXT,0,None,0
2,2,GID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of Language


...

----------------------------------------------------------------------------------------------------


Schema of M_Director


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,PID,TEXT,0,None,0
3,3,ID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of M_Cast


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,PID,TEXT,0,None,0
3,3,ID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------




## Useful tips:

1. the year column in 'Movie' table, will have few chracters other than numbers which you need to be preprocessed, you need to get a substring of last 4 characters, its better if you convert it as int type, ex: CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER)

2. For almost all the TEXT columns we have show, please try to remove trailing spaces, you need to use TRIM() function

3. When you are doing count(coulmn) it won't consider the "NULL" values, you might need to explore other alternatives like Count(*)

## Q1 --- List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.

<h4>To determine whether a year is a leap year, follow these steps:</h4>

<ul>
    <li><b>STEP-1:</b> If the year is evenly divisible by 4, go to step 2. Otherwise, go to step 5.</li>
    <li><b>STEP-2:</b> If the year is evenly divisible by 100, go to step 3. Otherwise, go to step 4.</li>
    <li><b>STEP-3:</b> If the year is evenly divisible by 400, go to step 4. Otherwise, go to step 5.</li>
    <li><b>STEP-4:</b> The year is a leap year (it has 366 days).</li>
    <li><b>STEP-5:</b> The year is not a leap year (it has 365 days).</li>
</ul>

Year 1900 is divisible by 4 and 100 but it is not divisible by 400, so it is not a leap year.

In [43]:
%%time
def grader_1(q1):
    q1_results  = pd.read_sql_query(q1,conn)
    print(q1_results.head(10))
    assert (q1_results.shape == (232,3))

query1 = """
        SELECT p.Name director,m.title movie_name,m.year FROM Movie m,Person p,M_Director md
        WHERE p.PID = md.PID
        AND md.MID =m.MID
        AND m.MID in
        (
          SELECT MID from M_Genre mg WHERE GID in
          (
            SELECT GID from Genre g WHERE Name like '%Comedy%'
          )
        )
        AND
        (
          (
            CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER)%4=0 AND CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER)%100<>0
          ) 
          OR CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER)%400=0
        )       
        """
grader_1(query1)

            director                         movie_name  year
0       Milap Zaveri                         Mastizaade  2016
1       Danny Leiner  Harold & Kumar Go to White Castle  2004
2     Anurag Kashyap                 Gangs of Wasseypur  2012
3       Frank Coraci        Around the World in 80 Days  2004
4      Griffin Dunne             The Accidental Husband  2008
5        Anurag Basu                             Barfi!  2012
6    Gurinder Chadha                  Bride & Prejudice  2004
7         Mike Judge    Beavis and Butt-Head Do America  1996
8   Tarun Mansukhani                            Dostana  2008
9       Shakun Batra                      Kapoor & Sons  2016
CPU times: user 58 ms, sys: 0 ns, total: 58 ms
Wall time: 59.7 ms


## Q2 --- List the names of all the actors who played in the movie 'Anand' (1971)

In [44]:
%%time
def grader_2(q2):
    q2_results  = pd.read_sql_query(q2,conn)
    print(q2_results.head(10))
    assert (q2_results.shape == (17,1))


query2 = """SELECT Name FROM Person p WHERE p.PID IN 
            (
              SELECT TRIM(PID) FROM M_Cast mc WHERE mc.MID in 
              (
                SELECT m.MID FROM Movie m WHERE title like 'Anand'
              )
            ) """
grader_2(query2)

                Name
0   Amitabh Bachchan
1      Rajesh Khanna
2      Sumita Sanyal
3         Ramesh Deo
4          Seema Deo
5     Asit Kumar Sen
6         Dev Kishan
7       Atam Prakash
8      Lalita Kumari
9             Savita
CPU times: user 29.2 ms, sys: 0 ns, total: 29.2 ms
Wall time: 26.2 ms


## Q3 --- List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

In [45]:
%%time

def grader_3a(query_less_1970, query_more_1990):
    q3_a = pd.read_sql_query(query_less_1970,conn)
    print(q3_a.shape)
    q3_b = pd.read_sql_query(query_more_1990,conn)
    print(q3_b.shape)
    return (q3_a.shape == (4942,1)) and (q3_b.shape == (62570,1))

query_less_1970 =""" 
Select p.PID from Person p 
inner join 
(
    select trim(mc.PID) PD, mc.MID from M_cast mc 
where mc.MID 
in 
(
    select mv.MID from Movie mv where CAST(SUBSTR(mv.year,-4) AS Integer)<1970
)
) r1 
on r1.PD=p.PID 
"""
query_more_1990 =""" 
Select p.PID from Person p 
inner join 
(
    select trim(mc.PID) PD, mc.MID from M_cast mc 
where mc.MID 
in 
(
    select mv.MID from Movie mv where CAST(SUBSTR(mv.year,-4) AS Integer)>1990
)
) r1 
on r1.PD=p.PID """
print(grader_3a(query_less_1970, query_more_1990))

# using the above two queries, you can find the answer to the given question 

(4942, 1)
(62570, 1)
True
CPU times: user 261 ms, sys: 2.9 ms, total: 264 ms
Wall time: 268 ms


In [46]:
%%time
def grader_3(q3):
    q3_results  = pd.read_sql_query(q3,conn)
    print(q3_results.head(10))
    assert (q3_results.shape == (300,1))

query3 ="""
        SELECT p.Name FROM Person p WHERE p.PID IN
        (
          SELECT TRIM(mca.PID) FROM M_Cast mca WHERE mca.MID IN
            (
              SELECT m.MID FROM Movie m WHERE CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER)<1970
            )
            AND mca.PID IN 
            (
              SELECT mc.PID FROM M_Cast mc WHERE mc.MID IN 
              (
                SELECT m.MID FROM Movie m WHERE CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER) > 1990
              )
            )

        )
        """
grader_3(query3)

                Name
0       Rishi Kapoor
1   Amitabh Bachchan
2             Asrani
3       Zohra Sehgal
4    Parikshat Sahni
5      Rakesh Sharma
6        Sanjay Dutt
7          Ric Young
8              Yusuf
9     Suhasini Mulay
CPU times: user 126 ms, sys: 791 µs, total: 127 ms
Wall time: 126 ms


## Q4 --- List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

In [47]:
%%time

def grader_4a(query_4a):
    query_4a = pd.read_sql_query(query_4a,conn)
    print(query_4a.head(10)) 
    return (query_4a.shape == (1462,2))

query_4a ="""SELECT md.PID,COUNT(MID) mid_count FROM M_Director md GROUP BY TRIM(md.PID)  """
print(grader_4a(query_4a))

# using the above query, you can write the answer to the given question

         PID  mid_count
0  nm0000180          1
1  nm0000187          1
2  nm0000229          1
3  nm0000269          1
4  nm0000386          1
5  nm0000487          2
6  nm0000965          1
7  nm0001060          1
8  nm0001162          1
9  nm0001241          1
True
CPU times: user 10.3 ms, sys: 34 µs, total: 10.4 ms
Wall time: 11.4 ms


In [48]:
%%time
def grader_4(q4):
    q4_results  = pd.read_sql_query(q4,conn)
    print(q4_results.head(10))
    assert (q4_results.shape == (58,2))

query4 ="""
        SELECT Name,COUNT(MID) mid_count FROM Person p JOIN M_Director md ON p.PID=md.PID GROUP BY md.PID HAVING mid_count >= 10 ORDER BY mid_count DESC
        """ 
grader_4(query4)

                    Name  mid_count
0           David Dhawan         39
1           Mahesh Bhatt         35
2           Priyadarshan         30
3        Ram Gopal Varma         30
4           Vikram Bhatt         29
5   Hrishikesh Mukherjee         27
6            Yash Chopra         21
7        Basu Chatterjee         19
8         Shakti Samanta         19
9           Subhash Ghai         18
CPU times: user 31.7 ms, sys: 0 ns, total: 31.7 ms
Wall time: 31.2 ms


## Q5.a --- For each year, count the number of movies in that year that had only female actors.

In [49]:
%%time

# note that you don't need TRIM for person table

def grader_5aa(query_5aa):
    query_5aa = pd.read_sql_query(query_5aa,conn)
    print(query_5aa.head(10)) 
    return (query_5aa.shape == (8846,3))

query_5aa ="""
              SELECT MID,Gender, COUNT(Gender) count FROM M_Cast mc JOIN Person p ON TRIM(mc.PID) = p.PID GROUP BY MID,GENDER
           """

print(grader_5aa(query_5aa))

def grader_5ab(query_5ab):
    query_5ab = pd.read_sql_query(query_5ab,conn)
    print(query_5ab.head(10)) 
    return (query_5ab.shape == (3469, 3))

query_5ab = """
              SELECT MID,Gender, COUNT(Gender) count FROM M_Cast mc JOIN Person p ON TRIM(mc.PID) = p.PID  GROUP BY MID,GENDER HAVING count>0 AND Gender='Male'  
            """

print(grader_5ab(query_5ab))


# using the above queries, you can write the answer to the given question

         MID  Gender  count
0  tt0021594    None      0
1  tt0021594  Female      3
2  tt0021594    Male      5
3  tt0026274    None      0
4  tt0026274  Female     11
5  tt0026274    Male      9
6  tt0027256    None      0
7  tt0027256  Female      5
8  tt0027256    Male      8
9  tt0028217  Female      3
True
         MID Gender  count
0  tt0021594   Male      5
1  tt0026274   Male      9
2  tt0027256   Male      8
3  tt0028217   Male      7
4  tt0031580   Male     27
5  tt0033616   Male     46
6  tt0036077   Male     11
7  tt0038491   Male      7
8  tt0039654   Male      6
9  tt0040067   Male     10
True
CPU times: user 315 ms, sys: 7.98 ms, total: 323 ms
Wall time: 320 ms


In [50]:
%%time
def grader_5a(q5a):
    q5a_results  = pd.read_sql_query(q5a,conn)
    print(q5a_results.head(10))
    assert (q5a_results.shape == (4,2))

query5a = """ SELECT year, COUNT(MID) FROM Movie m WHERE m.MID NOT IN
              (
                SELECT mc.MID FROM M_Cast mc JOIN Person p ON TRIM(mc.PID)=p.PID WHERE p.Gender ='Male'
              ) GROUP BY m.year """
grader_5a(query5a)

     year  COUNT(MID)
0    1939           1
1    1999           1
2    2000           1
3  I 2018           1
CPU times: user 139 ms, sys: 2.95 ms, total: 142 ms
Wall time: 139 ms


## Q5.b --- Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.

In [51]:
%%time
def grader_5b(q5b):
    q5b_results  = pd.read_sql_query(q5b,conn)
    print(q5b_results.head(10))
    assert (q5b_results.shape == (4,3))

query5b = """ SELECT ofc_year,CAST(only_female_count AS float)/all_movies_count*100  percentage_female_movies,all_movies_count FROM
              (
                SELECT CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER) ofc_year,COUNT(m.MID) only_female_count FROM Movie m WHERE m.MID NOT IN 
                (
                  SELECT mc.MID FROM M_Cast mc JOIN Person p ON TRIM(mc.PID)=p.PID WHERE p.Gender ='Male'
                ) 
              GROUP BY ofc_year
              ) only_female_table,
              (
                SELECT CAST(SUBSTR(TRIM(mn.year),-4) AS INTEGER) amc_year,count(mn.MID) all_movies_count FROM Movie mn GROUP BY amc_year
              ) all_movies
              WHERE amc_year=ofc_year 
            """
grader_5b(query5b)

   ofc_year  percentage_female_movies  all_movies_count
0      1939                 50.000000                 2
1      1999                  1.515152                66
2      2000                  1.562500                64
3      2018                  0.961538               104
CPU times: user 137 ms, sys: 1.97 ms, total: 139 ms
Wall time: 139 ms


## Q6 --- Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once.

In [52]:
%%time
def grader_6(q6):
    q6_results  = pd.read_sql_query(q6,conn)
    print(q6_results.head(10))
    assert (q6_results.shape == (3473, 2))

query6 = """SELECT title,cast_count FROM Movie,
            (
              SELECT MID,COUNT(PID) cast_count FROM 
              (
                SELECT DISTINCT MID ,PID FROM M_CAST
              ) dist_table
              GROUP BY MID
            )count_table
            WHERE count_table.MID=Movie.MID  ORDER BY cast_count DESC
        """
grader_6(query6)

                        title  cast_count
0               Ocean's Eight         238
1                    Apaharan         233
2                        Gold         215
3             My Name Is Khan         213
4  Captain America: Civil War         191
5                    Geostorm         170
6                     Striker         165
7                        2012         154
8                      Pixels         144
9       Yamla Pagla Deewana 2         140
CPU times: user 150 ms, sys: 8 ms, total: 158 ms
Wall time: 158 ms


### Q7 --- A decade is a sequence of 10 consecutive years. 
### For example, say in your database you have movie information starting from 1931. 
### the first decade is 1931, 1932, ..., 1940,
### the second decade is 1932, 1933, ..., 1941 and so on. 
### Find the decade D with the largest number of films and the total number of films in D

In [53]:
%%time
def grader_7a(q7a):
    q7a_results  = pd.read_sql_query(q7a,conn)
    print(q7a_results.head(10))
    assert (q7a_results.shape == (78, 2))

query7a = """ SELECT CAST(SUBSTR(TRIM(year),-4) AS INTEGER) y,count(MID) FROM Movie group by y """
grader_7a(query7a)

# using the above query, you can write the answer to the given question

      y  count(MID)
0  1931           1
1  1936           3
2  1939           2
3  1941           1
4  1943           1
5  1946           2
6  1947           2
7  1948           3
8  1949           3
9  1950           2
CPU times: user 5.85 ms, sys: 1.01 ms, total: 6.86 ms
Wall time: 6.73 ms


In [54]:
%%time
def grader_7b(q7b):
    q7b_results  = pd.read_sql_query(q7b,conn)
    print(q7b_results.head(10))
    assert (q7b_results.shape == (713, 4))

query7b ="""SELECT * FROM
            (
            SELECT CAST(SUBSTR(TRIM(year),-4) AS INTEGER) y,count(MID) FROM Movie group by y
            ) t1 JOIN
            (
            SELECT CAST(SUBSTR(TRIM(year),-4) AS INTEGER) y,count(MID) FROM Movie group by y
            ) t2
            ON t2.y <= t1.y + 9 AND t2.y >= t1.y
        """
          
grader_7b(query7b)
# if you see the below results the first movie year is less than 2nd movie year and 
# 2nd movie year is less or equal to the first movie year+9

# using the above query, you can write the answer to the given question

      y  count(MID)     y  count(MID)
0  1931           1  1931           1
1  1931           1  1936           3
2  1931           1  1939           2
3  1936           3  1936           3
4  1936           3  1939           2
5  1936           3  1941           1
6  1936           3  1943           1
7  1939           2  1939           2
8  1939           2  1941           1
9  1939           2  1943           1
CPU times: user 11.1 ms, sys: 2.05 ms, total: 13.2 ms
Wall time: 13.2 ms


In [55]:
%%time
def grader_7(q7):
    q7_results  = pd.read_sql_query(q7,conn)
    print(q7_results.head(10))
    assert (q7_results.shape == (1, 2))

query7 = """SELECT SUM(cm2) Decade_movie_count, GROUP_CONCAT(t2.y,'-') Decade FROM
            (
              SELECT CAST(SUBSTR(TRIM(year),-4) AS INTEGER) y,count(MID) cm1 FROM Movie group by y
            ) t1 JOIN
            (
              SELECT CAST(SUBSTR(TRIM(year),-4) AS INTEGER) y,count(MID) cm2 FROM Movie group by y
            ) t2
            ON t2.y <= t1.y + 9 AND t2.y >= t1.y GROUP BY t1.y ORDER BY Decade_movie_count DESC LIMIT 1
        """
grader_7(query7)
# if you check the output we are printinng all the year in that decade, its fine you can print 2008 or 2008-2017

   Decade_movie_count                                             Decade
0                1203  2008-2009-2010-2011-2012-2013-2014-2015-2016-2017
CPU times: user 9.1 ms, sys: 0 ns, total: 9.1 ms
Wall time: 13.8 ms


## Q8 --- Find all the actors that made more movies with Yash Chopra than any other director.

In [56]:
%%time
def grader_8a(q8a):
    q8a_results  = pd.read_sql_query(q8a,conn)
    print(q8a_results.head(10))
    assert (q8a_results.shape == (73408, 3))

query8a = """ SELECT TRIM(mc.PID) actor,md.PID director, COUNT(*) movies FROM M_Cast mc, M_Director md WHERE mc.MID = md.MID  GROUP BY md.PID,actor"""
grader_8a(query8a)

# using the above query, you can write the answer to the given question

       actor   director  movies
0  nm0000027  nm0000180       1
1  nm0001114  nm0000180       1
2  nm0001919  nm0000180       1
3  nm0006762  nm0000180       1
4  nm0030062  nm0000180       1
5  nm0038970  nm0000180       1
6  nm0051856  nm0000180       1
7  nm0085966  nm0000180       1
8  nm0097889  nm0000180       1
9  nm0125497  nm0000180       1
CPU times: user 284 ms, sys: 12 ms, total: 296 ms
Wall time: 299 ms


In [57]:
%%time

def grader_8(q8):
    q8_results  = pd.read_sql_query(q8,conn)
    print(q8_results.head(10))
    print(q8_results.shape)
    assert (q8_results.shape == (245, 2))

query8 = """SELECT p.Name,final.movies FROM Person p,
            (
              SELECT yash_movies.actor actor,yash_movies.movies movies FROM 
              (  
                SELECT t2.yash,t1.director,t1.actor actor,t1.movies movies FROM
                (
                  SELECT p.PID yash FROM Person p WHERE p.Name LIKE '%Yash Chopra%' 
                ) t2,
                (  
                  SELECT TRIM(mc.PID) actor,TRIM(md.PID) director, COUNT(*) movies FROM M_Cast mc, M_Director md WHERE mc.MID = md.MID  GROUP BY actor, director
                ) t1
                WHERE t2.yash=t1.director
              )yash_movies
              WHERE yash_movies.actor NOT IN
              (
                SELECT DISTINCT yash_movies.actor FROM
                (  
                  SELECT t2.yash,t1.director,t1.actor actor,t1.movies movies FROM
                  (
                    SELECT p.PID yash FROM Person p WHERE p.Name LIKE '%Yash Chopra%' 
                  ) t2,
                  (  
                    SELECT TRIM(mc.PID) actor,TRIM(md.PID) director, COUNT(*) movies FROM M_Cast mc, M_Director md WHERE mc.MID = md.MID  GROUP BY actor, director
                  ) t1
                  WHERE t2.yash=t1.director
                )yash_movies,
                (
                  SELECT TRIM(mc.PID) actor,TRIM(md.PID) director, COUNT(*) movies FROM M_Cast mc, M_Director md WHERE mc.MID = md.MID  GROUP BY actor, director
                )all_dir
                WHERE  yash_movies.actor=all_dir.actor AND yash_movies.movies<all_dir.movies
              )
            )final 
            WHERE p.PID= final.actor ORDER BY final.movies DESC

      """
grader_8(query8)

                Name  movies
0        Jagdish Raj      11
1   Manmohan Krishna      10
2           Iftekhar       9
3      Shashi Kapoor       7
4      Rakhee Gulzar       5
5     Waheeda Rehman       5
6           Ravikant       4
7     Achala Sachdev       4
8        Neetu Singh       4
9      Leela Chitnis       3
(245, 2)
CPU times: user 939 ms, sys: 27.1 ms, total: 966 ms
Wall time: 968 ms


## Q9 --- The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.

In [58]:
%%time
def grader_9a(q9a):
    q9a_results  = pd.read_sql_query(q9a,conn)
    print(q9a_results.head(10))
    print(q9a_results.shape)
    assert (q9a_results.shape == (2382, 1))

query9a = """
      SELECT DISTINCT TRIM(mc.PID) actor FROM M_Cast mc,
      (
        SELECT mc.MID,khan_id FROM M_Cast mc,
        (
          SELECT p.PID khan_id FROM Person p WHERE p.Name LIKE '%Shah Rukh Khan%'
        )
        WHERE TRIM(mc.PID)=khan_id
      )khan_movies  
      WHERE mc.MID = khan_movies.MID AND actor != khan_id
      """
grader_9a(query9a)
# using the above query, you can write the answer to the given question

# selecting actors who acted with srk (S1)
# selecting all movies where S1 actors acted, this forms S2 movies list
# selecting all actors who acted in S2 movies, this gives us S2 actors along with S1 actors
# removing S1 actors from the combined list of S1 & S2 actors, so that we get only S2 actors

       actor
0  nm0004418
1  nm1995953
2  nm2778261
3  nm0631373
4  nm0241935
5  nm0792116
6  nm1300111
7  nm0196375
8  nm1464837
9  nm2868019
(2382, 1)
CPU times: user 3.09 s, sys: 16.2 ms, total: 3.1 s
Wall time: 3.08 s


In [59]:
%%time
def grader_9(q9):
    q9_results  = pd.read_sql_query(q9,conn)
    print(q9_results.head(10))
    print(q9_results.shape)
    assert (q9_results.shape == (25698, 1))

query9 = """
      SELECT p.Name Actor_Name FROM Person p,
      (
        SELECT DISTINCT TRIM(mc.PID) actor2 FROM M_Cast mc,
        (
          SELECT mc.MID FROM M_Cast mc,
          (
            SELECT DISTINCT TRIM(mc.PID) actor FROM M_Cast mc,
            (
              SELECT mc.MID,khan_id FROM M_Cast mc,
              (
                SELECT p.PID khan_id FROM Person p WHERE p.Name LIKE '%Shah Rukh Khan%'
              )
              WHERE TRIM(mc.PID)=khan_id
            )khan_movies  
            WHERE mc.MID = khan_movies.MID AND actor != khan_id
          )khan1actors
          WHERE khan1actors.actor = TRIM(mc.PID)
        )khan1movies
        WHERE khan1movies.MID = mc.MID AND actor2 NOT IN
        (
          SELECT DISTINCT TRIM(mc.PID) actor FROM M_Cast mc,
          (
            SELECT mc.MID,khan_id FROM M_Cast mc,
            (
              SELECT p.PID khan_id FROM Person p WHERE p.Name LIKE '%Shah Rukh Khan%'
            )
            WHERE TRIM(mc.PID)=khan_id
          )khan_movies  
          WHERE mc.MID = khan_movies.MID AND actor != khan_id
        ) 
        AND actor2 NOT IN 
        (
          SELECT p.PID khan_id FROM Person p WHERE p.Name LIKE '%Shah Rukh Khan%'
        )
      )khan2actors
      WHERE khan2actors.actor2=p.PID
      """
grader_9(query9)

               Actor_Name
0            Freida Pinto
1             Rohan Chand
2            Damian Young
3         Waris Ahluwalia
4   Caroline Christl Long
5           Rajeev Pahuja
6       Michelle Santiago
7         Alicia Vikander
8            Dominic West
9          Walton Goggins
(25698, 1)
CPU times: user 7.78 s, sys: 52.4 ms, total: 7.83 s
Wall time: 7.81 s
